In [189]:
import pandas as pd
import numpy as np
import plotly as px
import matplotlib.pyplot as plt
from colorama import Fore, Back, Style
import json

# Utils

In [190]:
def get_maires_dict() -> dict:
    
    mairesDict = {
    "Stephane JAY" : [1896, 1904],
    "Charles RIVAIL" : [1904, 1908],
    "Felix VIALLET" : [1908, 1910],
    "Nestor CORNIER" : [1910, 1919],
    "Paul MISTRAL" : [1919, 1932],
    "Leon MARTIN" : [1932, 1935],
    "Paul COCAT" : [1935, 1944],
    "Frederic LAFLEUR" : [1944, 1945],
    "Leon MARTIN" : [1945, 1947],
    "Marius BALLY" : [1947, 1948],
    "Remy PERINETTI" : [1948, 1949],
    "Leon MARTIN" : [1949, 1959],
    "Albert MICHALLON" : [1959, 1965],
    "Hubert DUBEDOUT" : [1965, 1983],
    "Alain CARIGNON" : [1983, 1995],
    "Michel DESTOT" : [1995, 2014],
    "Eric PIOLLE" : [2014, 2021],
}
    return mairesDict

In [191]:
# Couleurs diverses
VERT='\033[0;32m' # vert mince
BLANC='\033[1;37m' # blanc gras
DEFAUT='\033[0;m' # couleur console par defaut
NOIR='\033[0;30m' # noir mince
ROUGE='\033[0;31m' # rouge mince
MARRON='\033[0;33m' # marron mince
BLEU='\033[0;34m' # bleu fonce mince
VIOLET='\033[0;35m' # violet mince
CYAN='\033[0;36m' # cyan mince
GRIS='\033[0;37m' # gris clair mince
BLANCLEGER='\033[0;38m' # blanc mince
ROUGECLAIR='\033[1;31m' # rouge clair gras
VERTCLAIR='\033[1;32m' # vert clair gras
JAUNE='\033[1;33m' # jaune gras
BLEUCLAIR='\033[1;34m' # bleu clair gras
ROSE='\033[1;35m' # rose clair gras
CYANCLAIR='\033[1;36m' # cyan clair gras

In [192]:
def change_column_name(df:pd.DataFrame) -> pd.DataFrame:
    valid = {"yes": True, "y": True, "ye": True, "Yes":True, "no": False, "n": False, "No": False}
    newNameList = []
    for col in df.columns.tolist():
        print(CYAN, "Do you want do rename the column", ROUGECLAIR, col,CYAN, "y / n")
        res = input()
        
        if res in list(valid.keys()):
        
            if valid[res] == True:
                print(ROUGECLAIR, "New name....")
                newName = input().upper()
                newNameList.append(newName)

            elif valid[res] == False:
                newNameList.append(col)
        else:
            print(ROUGECLAIR, "You exit the program")
            return df
        
    print(VERTCLAIR, newNameList)    
    df.columns = newNameList
    return df

# I - Import data

In [193]:
# This csv is imported with a value added in the last ligne
# There was originally a missing value
# Here in a csv : In last rows / col 1 : 30032 was added
file_path = '../data/ESP_PUBLIC.IDENTITE_ARBRE.csv'
df = pd.read_csv(file_path, sep=',', header = 0, index_col=False)
dff = df

# II - Computations

##### Missing years for plantation

In [194]:
# Convert the column year from FLOAT --> INT
# --> Not working because of Nan : dff["ANNEEDEPLANTATION"].astype("int")
missingYear = dff["ANNEEDEPLANTATION"].isnull().value_counts()[1]
numberOfTree = dff["ANNEEDEPLANTATION"].size

print(CYANCLAIR, "Total of Trees -->", BLEUCLAIR, numberOfTree, \
      CYANCLAIR, "<-- with total of -->", BLEUCLAIR, missingYear,\
      CYANCLAIR, "<-- missing years")

 Total of Trees -->  31619  <-- with total of -->  1438  <-- missing years


##### Total of trees for specific year

In [195]:
# First replace the .nan by float(0)
# After that convert float to int
dff["ANNEEDEPLANTATION"].fillna(0).astype("int")

0        1977
1        2001
2        1975
3        1978
4        1960
         ... 
31614    2014
31615    2014
31616    2014
31617    2014
31618    1995
Name: ANNEEDEPLANTATION, Length: 31619, dtype: int64

In [196]:
firstYearPlanted = int(dff["ANNEEDEPLANTATION"].min())
lastYearPlanted = int(dff["ANNEEDEPLANTATION"].max())
print(CYANCLAIR, "The first tree was planted in -->", BLEUCLAIR, firstYearPlanted)
print(CYANCLAIR, "The last tree was planted in --->", BLEUCLAIR, lastYearPlanted)

 The first tree was planted in -->  1900
 The last tree was planted in --->  2021


##### Study by years

In [197]:
df_years = dff[["ANNEEDEPLANTATION"]]
df_years.value_counts()

ANNEEDEPLANTATION
1975.0               3631
2001.0               1488
1980.0               1445
1970.0               1366
2015.0               1012
                     ... 
1959.0                 16
1924.0                 11
1964.0                  6
1900.0                  4
2021.0                  3
Length: 65, dtype: int64

In [198]:
for key in get_maires_dict().keys():
    print(CYAN, key, ROUGE, get_maires_dict()[key])

 Stephane JAY  [1896, 1904]
 Charles RIVAIL  [1904, 1908]
 Felix VIALLET  [1908, 1910]
 Nestor CORNIER  [1910, 1919]
 Paul MISTRAL  [1919, 1932]
 Leon MARTIN  [1949, 1959]
 Paul COCAT  [1935, 1944]
 Frederic LAFLEUR  [1944, 1945]
 Marius BALLY  [1947, 1948]
 Remy PERINETTI  [1948, 1949]
 Albert MICHALLON  [1959, 1965]
 Hubert DUBEDOUT  [1965, 1983]
 Alain CARIGNON  [1983, 1995]
 Michel DESTOT  [1995, 2014]
 Eric PIOLLE  [2014, 2021]


##### Create a list of tuple from 2 pandas Series

In [199]:
df_tuple = pd.DataFrame()
df_tuple['year_genre'] = list(zip(dff["ANNEEDEPLANTATION"], dff["GENRE_BOTA"]))

# III - Maire study

In [200]:
def find_maire(year:int) -> str:
    mairesDict = get_maires_dict()
    if year:
        for key in mairesDict.keys():
            if int(mairesDict[key][0]) <= int(year) < int(mairesDict[key][1]):
                return key
        
    return np.nan
dff["MAIRE"] = dff["ANNEEDEPLANTATION"].dropna().apply(find_maire)

In [201]:
# Extract only the four usefull column
df_maire = dff[["MAIRE", "ANNEEDEPLANTATION", "GENRE_BOTA", "ESPECE"]]

In [202]:
# df_maire = change_column_name(df_maire)

##### Fin the maire who planted the more trees

In [203]:
df_maire["MAIRE"].value_counts()

Hubert DUBEDOUT     12074
Michel DESTOT        8554
Alain CARIGNON       4264
Eric PIOLLE          4001
Albert MICHALLON      723
Leon MARTIN           520
Frederic LAFLEUR       27
Paul MISTRAL           11
Stephane JAY            4
Name: MAIRE, dtype: int64

In [204]:
maxMaireName = df_maire["MAIRE"].value_counts().idxmax()
maxMaireTree = df_maire["MAIRE"].value_counts().max()
print(ROUGECLAIR, "the mayor -->", \
      CYAN, maxMaireName, ROUGECLAIR, "<-- planted the more tree with a total of -->", \
     CYAN, maxMaireTree, ROUGECLAIR, "<--trees")

 the mayor -->  Hubert DUBEDOUT  <-- planted the more tree with a total of -->  12074  <--trees


##### Botanic and Species

In [205]:
df_maire["GENRE_BOTA"].value_counts()

Acer           5055
Platanus       4492
Pinus          1962
Tilia          1740
Fraxinus       1505
               ... 
Castanea          1
Eucalyptus        1
Poncinos          1
Sciadopitys       1
Nyssa             1
Name: GENRE_BOTA, Length: 119, dtype: int64

In [206]:
df_maire.groupby(["MAIRE"])[["GENRE_BOTA", "ESPECE"]].value_counts().reset_index()

,MAIRE,GENRE_BOTA,ESPECE,0
0,Alain CARIGNON,Platanus,acerifolia,606
1,Alain CARIGNON,Carpinus,betulus,366
2,Alain CARIGNON,Liquidambar,styraciflua,299
3,Alain CARIGNON,Pinus,nigra,176
4,Alain CARIGNON,Tilia,tomentosa,147
...,...,...,...,...
828,Paul MISTRAL,Olea,europaea,1
829,Paul MISTRAL,Picea,abies,1
830,Paul MISTRAL,Chamaecyparis,formosensis,1
831,Paul MISTRAL,Betula,verrucosa,1


In [207]:
df_maire[df_maire["MAIRE"] == "Hubert DUBEDOUT"]["MAIRE"].shape

(12074,)

In [208]:
df_maire[df_maire["MAIRE"] == "Hubert DUBEDOUT"]["ANNEEDEPLANTATION"].astype("int")

0        1977
2        1975
3        1978
11       1980
13       1982
         ... 
31601    1970
31602    1970
31603    1970
31604    1970
31605    1970
Name: ANNEEDEPLANTATION, Length: 12074, dtype: int64

#  IV - Start Iteration 3